### Classifying and predicting early hospital readmissions:

Classifying with supervised learning whether diabetic patients are readmitted, and if they are, if it's before or after 30 days.

Using the dataset from here: https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008

In [23]:
import pandas as pd
import patsy as patsy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing

# SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Undersampling
from imblearn.under_sampling import RandomUnderSampler

from sklearn import metrics
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler

import pickle

# models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import __version__ as sklearnver
print(sklearnver)

0.21.2


In [24]:
with open("x_liv.pkl", 'rb') as picklefile:
    X = pickle.load(picklefile)

with open("y_liv.pkl", 'rb') as picklefile:
    y = pickle.load(picklefile)

In [25]:
y = y.replace({'<30': 1, '>30': 0, 'NO': 0})

In [26]:
# y.head()
# set(list(y))
# y.describe()

In [27]:
# stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [28]:
# standard scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
# # smote oversampling (now moved to pipeline)
# X_train, y_train = smote.fit_sample(X_train, y_train)

In [30]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
# models.append(('KNN', KNeighborsClassifier())) # can take a very long time
models.append(('DT', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('ETC', ExtraTreesClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('ADA', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))
# models.append(('SVM', SVC())) # can take a very long time
# evaluate each model in turn
results = []
names = []
scoring = 'roc_auc'

In [31]:
kfold = StratifiedKFold(n_splits=5, random_state=42)
smote = SMOTE()

for name, model in models:
    pipeline = Pipeline([('smote', smote),
                     (name,model)])
    cv_results = cross_val_score(
        pipeline, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

KeyboardInterrupt: 

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10,10))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
# store model score rather than just printing
# best_model_name = %% sort through the tuples or dictionary to find the best model
# print the name of the best model

In [33]:
# Best model was extra trees:

best_model_name = 'GBM' # manual assignment will be replaced with above code block

if best_model_name == 'GBM':

    param_grid = {
    f"{best_model_name}__learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    f"{best_model_name}__min_samples_split": np.linspace(0.1, 0.5, 12),
    f"{best_model_name}__min_samples_leaf": np.linspace(0.1, 0.5, 12),
    f"{best_model_name}__max_depth":[3,5,8],
    f"{best_model_name}__max_features":["log2","sqrt"],
    f"{best_model_name}__criterion": ["friedman_mse",  "mae"],
    f"{best_model_name}__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    f"{best_model_name}__n_estimators":[10,100,250,500],
    f"{best_model_name}__n_iter_no_change":[10],
    f"{best_model_name}__random_state":[42],
    }

    model = GradientBoostingClassifier()

if best_model_name == 'RF':
    param_grid = {f'{best_model_name}__n_estimators': range(50, 201, 25),
                  f'{best_model_name}__max_features': range(50, X.shape[1], 50),
                  f'{best_model_name}__min_samples_leaf': range(20, 50, 5),
                  f'{best_model_name}__min_samples_split': range(15, 36, 5)}

    model = ExtraTreesClassifier(n_estimators=100, n_jobs=-1, min_samples_split=25,
                                 min_samples_leaf=35, max_features=150)

In [34]:
kfold = StratifiedKFold(n_splits=5, random_state=42)

In [35]:
pipeline = Pipeline([('smote', smote),
         (best_model_name, model)])

def grid_or_random_cv(grid_or_random='random'):
    if grid_or_random == 'grid':
        grid_search = GridSearchCV(estimator=pipeline,
                                   param_grid=param_grid,
                                   scoring='roc_auc',
                                   cv=kfold)

        grid_result = grid_search.fit(X_train, y_train)

    if grid_or_random == 'random':
        n_iter_search = 25
        random_search = RandomizedSearchCV(estimator=pipeline,
                                           param_distributions=param_grid,
                                           scoring='roc_auc',
                                           n_iter=n_iter_search,
                                           cv=kfold)

        grid_result = random_search.fit(X_train, y_train)

    return grid_result

In [36]:
grid_result = grid_or_random_cv(grid_or_random='random')

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.617660 using {'GBM__subsample': 0.9, 'GBM__random_state': 42, 'GBM__n_iter_no_change': 10, 'GBM__n_estimators': 100, 'GBM__min_samples_split': 0.46363636363636374, 'GBM__min_samples_leaf': 0.42727272727272736, 'GBM__max_features': 'sqrt', 'GBM__max_depth': 3, 'GBM__learning_rate': 0.05, 'GBM__criterion': 'friedman_mse'}


In [37]:
for i, (test_mean, test_std, param) in enumerate(zip(
        grid_result.cv_results_['mean_test_score'],
        grid_result.cv_results_['std_test_score'],
        grid_result.cv_results_['params'])):
    print("%i Test avg: %f \n Test std: %f with: \n %r" %
          (i, test_mean, test_std, param))

0 Test avg: 0.500000 
 Test std: 0.000000 with: 
 {'GBM__subsample': 1.0, 'GBM__random_state': 42, 'GBM__n_iter_no_change': 10, 'GBM__n_estimators': 250, 'GBM__min_samples_split': 0.28181818181818186, 'GBM__min_samples_leaf': 0.5, 'GBM__max_features': 'log2', 'GBM__max_depth': 5, 'GBM__learning_rate': 0.2, 'GBM__criterion': 'mae'}
1 Test avg: 0.598950 
 Test std: 0.004625 with: 
 {'GBM__subsample': 0.8, 'GBM__random_state': 42, 'GBM__n_iter_no_change': 10, 'GBM__n_estimators': 100, 'GBM__min_samples_split': 0.1, 'GBM__min_samples_leaf': 0.390909090909091, 'GBM__max_features': 'sqrt', 'GBM__max_depth': 8, 'GBM__learning_rate': 0.1, 'GBM__criterion': 'mae'}
2 Test avg: 0.603490 
 Test std: 0.006988 with: 
 {'GBM__subsample': 0.9, 'GBM__random_state': 42, 'GBM__n_iter_no_change': 10, 'GBM__n_estimators': 100, 'GBM__min_samples_split': 0.46363636363636374, 'GBM__min_samples_leaf': 0.390909090909091, 'GBM__max_features': 'log2', 'GBM__max_depth': 8, 'GBM__learning_rate': 0.01, 'GBM__criteri

In [38]:
model = ExtraTreesClassifier(**grid_result.best_params_)

TypeError: __init__() got an unexpected keyword argument 'GBM__subsample'

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_proba = model.predict_proba(X_test)

In [ ]:
# import plots

In [ ]:
print(metrics.classification_report(y_test, y_pred))

In [ ]:
def precision_recall_plot(y_test, y_pred_proba):
    p, r, t = precision_recall_curve(y_test, y_pred_proba[:, 1])

    # adding last threshold of '1' to threshold list
    t = np.vstack([t.reshape([-1, 1]), 1])
    
    # boxplot algorithm comparison
    fig = plt.figure(figsize=(10,10))
    fig.suptitle('Precision Recall Curve')
    ax = fig.add_subplot(111)
    plt.plot(t, p, label="precision")
    plt.plot(t, r, label="recall")
    plt.show()

    return fig

In [ ]:
precision_recall_plot(y_test, y_pred_proba)

In [ ]:
class_names = ["not early readmit", "early readmit"]

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=sns.color_palette("Blues")):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Only use the labels that appear in the data
#     classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    ax = sns.heatmap(cm, cmap = cmap, annot=True, xticklabels=classes, yticklabels=classes)

    ax.set(title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the x labels and set their alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Vertically center y labels
    plt.setp(ax.get_yticklabels(), va="center")
    
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    
    return ax

In [ ]:
# Plot non-normalized confusion matrix
ax = plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')

# plt.savefig('confusion_matrix_test.png', bbox_inches="tight")

In [ ]:
model.feature_importances_

Combining grid search and model selection:

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV


class EstimatorSelectionHelper:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, **grid_kwargs):
        for key in self.keys:
            print('Running GridSearchCV for %s.' % key)
            model = self.models[key]
            params = self.params[key]
            grid_search = GridSearchCV(model, params, **grid_kwargs)
            grid_search.fit(X, y)
            self.grid_searches[key] = grid_search
        print('Done.')

    def score_summary(self, sort_by='mean_test_score'):
        frames = []
        for name, grid_search in self.grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
        df = pd.concat(frames)

        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_score', 'index'], 1)

        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df

In [ ]:
models = {
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

params = {
    'ExtraTreesClassifier': {'n_estimators': [16, 32]},
    'RandomForestClassifier': [
        {'n_estimators': [16, 32]},
        {'criterion': ['gini', 'entropy'], 'n_estimators': [8, 16]}],
    'AdaBoostClassifier':  {'n_estimators': [16, 32]},
    'GradientBoostingClassifier': {'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0]}
}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y)

In [ ]:
helper = EstimatorSelectionHelper(models, params)
helper.fit(X, y, scoring='f1', n_jobs=-1)
helper.fit(X, y, scoring='neg_log_loss', n_jobs=-1)
helper.score_summary()